In [13]:
import pandas as pd
dist_file = "월별_가입이력_dummy_data2.csv"
coverage_dist_df = pd.read_csv(dist_file)
df_cust = coverage_dist_df.copy()

In [14]:
dist_file = "고객가입담보이력.csv"
df_enroll_history = pd.read_csv(dist_file)

In [2]:
import os
from glob import glob
import streamlit as st
from loguru import logger
from openai import OpenAIError
from dotenv import load_dotenv
import warnings
import traceback  # 추가된 부분

# ✅ LangGraph 및 LangChain 관련 모듈
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

# ✅ AI Assistant LangGraph Class Import
from utils.ai_agent import AIAnalysisAssistant  # ai_agent.py에 해당 클래스 정의

warnings.filterwarnings('ignore')
load_dotenv()  # 환경 변수 로드

# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key)


In [44]:
results  = {}
prompt = f"""
제공된 하나 이상의 데이터마트의 구조를 이해하기 위해, 아래의 '데이터 파악 함수'를 활용하여 각 데이터의 전반적인 정보를 분석하는 Python 코드를 생성해주세요.

**요구사항:**
1. **다중 데이터프레임**을 분석해야 합니다. 데이터프레임 목록이 주어질 것이며, 각 데이터프레임에 대해 동일한 분석을 수행해야 합니다.
2. 각 데이터프레임은 **`summarize_data` 함수**를 사용하여 분석하고, 결과는 반드시 아래와 같은 형식으로 저장해야 합니다:
   - 데이터프레임 이름이 `df_mart1`이면 결과는 `df_mart1_rslt1`, `df_mart1_rslt2`, `df_mart1_rslt3`로 저장합니다.
   - 데이터프레임 이름이 `df_mart2`이면 결과는 `df_mart2_rslt1`, `df_mart2_rslt2`, `df_mart2_rslt3`로 저장합니다.
   - 이런 방식으로 모든 데이터프레임에 적용해주세요.

3. 분석할 데이터프레임 목록은 `dataframe_list`에 저장되어 있으며, **`analyze_multiple_dataframes` 함수**를 통해 일괄적으로 처리합니다.
4. **`print()`는 사용하지 않으며**, 결과값은 반환(return) 방식으로 처리해야 합니다.

**예시:**
```python
dataframe_list = ['df_mart1', 'df_mart2', 'df_mart3']
results = analyze_multiple_dataframes(dataframe_list)

# 결과 접근 예시
df_mart1_rslt1 = results.get('df_mart1_rslt1')
df_mart1_rslt2 = results.get('df_mart1_rslt2')
df_mart1_rslt3 = results.get('df_mart1_rslt3')
"""

func_code = open(f'func_data_summary.py', 'r', encoding='utf-8').read()
chain = ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("user","### 데이터 파악 함수:\n{func_code}\n\n"),
            ("user","### list_df:\n{list_df}\n\n")
        ]) | llm
response = chain.invoke({"func_code" : func_code, "list_df" : ["df_cust", "df_enroll_history"]}).content

if "```python" in response:
    modified_code = response.split("```python")[-1].split("```")[0]
# print(modified_code)
exec(modified_code, globals())



In [46]:
results['df_cust_rslt1'] 

,항목,내용
0,"전체 데이터 크기 (행, 열)","(120000, 123)"
1,전체 컬럼 수,123
2,전체 인스턴스 수 (레코드 수),120000
3,결측치 총 개수,1168224
4,결측치 비율(%),7.91
5,결측치가 있는 컬럼,"{'당월보험료자동대출잔액': 109962, 'CB신용평점': 105882, 'CB신..."
6,범주형 변수 수,5
7,연속형 변수 수,118
8,범주형 변수 목록,"[나이, 성별, 운전코드명, 성별코드, 직업분류명]"
9,연속형 변수 목록,"[고객ID, 수익자여부, CB신용평점, CB신용등급, 두낫콜여부, 피보험자여부, 보..."


# 1안

In [49]:
import sys, os
prompt = f"""
다음의 결측치 처리 함수를 참고해서, 제공된 df_mart 데이터프레임의 결측치를 처리한 Python 코드를 작성해줘.
임의의 예시 데이터프레임을 생성하지 말고, df_mart라는 데이터프레임이 이미 제공된 것으로 가정하고 코드를 작성해.
dict_json은 제외할 변수 목록이 비어있는 형태()로 사용해.
결측치 처리가 완료된 df_mart 데이터프레임과 결측치 처리 정보를 담은 df_missing_info 데이터프레임을 출력하는 코드를 작성해.
결측치 처리가 완료된 데이터프레임의 이름은 반드시 **df_mart**로 유지해야 해.
"""
func_code = open(f'func_pps_missing.py', 'r', encoding='utf-8').read()

chain = ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("user","### 데이터프레임:\n{dataframe}\n\n"),
            ("user","### 결측치 처리 함수:\n{func_code}\n\n")
        ]) | llm
response = chain.invoke({"dataframe": "df_mart", "func_code" : func_code}).content

if "```python" in response:
    modified_code = response.split("```python")[-1].split("```")[0]
exec(modified_code, globals())

   연속형1  연속형2 범주형1 범주형2  제외연속형 제외범주형
0   1.0   0.0    A   기타    1.0     A
1   2.0   2.5   기타    X    2.0    기타
2   0.0   3.5    B    Y    0.0     B
3   4.0   0.0    C   기타    4.0     C
     변수명  변수개수 처리방법
0   연속형1     1    0
1   연속형2     2    0
2   범주형1     1   기타
3   범주형2     2   기타
4  제외연속형     1    0
5  제외범주형     1   기타


In [50]:
print(modified_code)


import pandas as pd

# 결측치 처리 함수 정의
def func_pps_missing(df_mart_1, dict_json):   
    # 변수 리스트 생성
    df_var_list = pd.DataFrame(df_mart_1.dtypes, columns=["변수유형"]).reset_index().rename(columns={"index": "변수명"})

    # 연속형 변수 리스트 생성
    df_num_list = (
        df_var_list[
             (df_var_list["변수유형"] != 'object')
             & ~(df_var_list["변수명"].isin(dict_json.get("v_exc_num", [])))
        ]
    )    
         
    # 범주형 변수 리스트 생성
    df_char_list = (
        df_var_list[
                (df_var_list["변수유형"] == 'object')
            & ~(df_var_list["변수명"].isin(dict_json.get("v_exc_char", [])))
        ]
    )
    
    # 연속형 변수 결측값에 대해 일괄적으로 0으로 대체, 범주형 변수 결측값에 대해 일괄적으로 '기타'로 대체
    df_mart_2 = df_mart_1.copy()
    df_mart_2[df_num_list["변수명"]] = df_mart_2[df_num_list["변수명"]].fillna(0)
    df_mart_2[df_char_list["변수명"]] = df_mart_2[df_char_list["변수명"]].fillna('기타')
    
    # 결측치 변수 리스트 생성
    df_missing_list = pd.DataFrame(df_mart_1.isnull().sum(), columns=["변수개수"]).reset_i

# 2안

In [41]:
prompt = f"""
아래의 func_pps_missing 함수를 그대로 사용해서 제공된 df_mart 데이터프레임의 결측치를 처리한 Python 코드를 작성해줘.
임의의 예시 데이터프레임을 생성하지 말고, df_mart라는 데이터프레임이 이미 제공된 것으로 가정하고 코드를 작성해.
dict_json은 제외할 변수 목록이 비어있는 형태()로 사용해.
결측치 처리가 완료된 df_mart 데이터프레임과 결측치 처리 정보를 담은 df_missing_info 데이터프레임을 출력하는 코드를 작성해.
결측치 처리가 완료된 데이터프레임의 이름은 반드시 **df_mart**로 유지해야 해.
"""
func_code = open(f'func_pps_missing.py', 'r', encoding='utf-8').read()

chain = ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("user","### 데이터프레임:\n{dataframe}\n\n"),
            ("user","### 결측치 처리 함수:\n{func_code}\n\n")
        ]) | llm
response = chain.invoke({"dataframe": "df_mart", "func_code" : func_code}).content

if "```python" in response:
    modified_code = response.split("```python")[-1].split("```")[0]
exec(modified_code, globals())

결측치 처리 완료된 df_mart:
     A   B    C      D
0  1.0   x  0.0    cat
1  2.0  기타  1.5    dog
2  0.0   z  2.5     기타
3  4.0   y  0.0  mouse

결측치 처리 정보:
  변수명  변수개수 처리방법
0   A     1    0
1   B     1   기타
2   C     2    0
3   D     1   기타


In [42]:
print(modified_code)


import pandas as pd

# 결측치 처리 함수 정의
def func_pps_missing(df_mart_1, dict_json):   
    # 변수 리스트 생성
    df_var_list = pd.DataFrame(df_mart_1.dtypes, columns=["변수유형"]).reset_index().rename(columns={"index": "변수명"})

    # 연속형 변수 리스트 생성
    df_num_list = (
        df_var_list[
            (df_var_list["변수유형"] != 'object')
            & ~(df_var_list["변수명"].isin(dict_json.get("v_exc_num", [])))
        ]
    )    
         
    # 범주형 변수 리스트 생성
    df_char_list = (
        df_var_list[
            (df_var_list["변수유형"] == 'object')
            & ~(df_var_list["변수명"].isin(dict_json.get("v_exc_char", [])))
        ]
    )
    
    # 연속형 변수 결측값에 대해 일괄적으로 0으로 대체, 범주형 변수 결측값에 대해 일괄적으로 '기타'로 대체
    df_mart_2 = df_mart_1.copy()
    df_mart_2[df_num_list["변수명"]] = df_mart_2[df_num_list["변수명"]].fillna(0)
    df_mart_2[df_char_list["변수명"]] = df_mart_2[df_char_list["변수명"]].fillna('기타')
    
    # 결측치 변수 리스트 생성
    df_missing_list = pd.DataFrame(df_mart_1.isnull().sum(), columns=["변수개수"]).reset_index()

In [46]:
import pandas as pd

# 결측치 처리 함수 정의
def func_pps_missing(df_mart_1, dict_json):   
    # 변수 리스트 생성
    df_var_list = pd.DataFrame(df_mart_1.dtypes, columns=["변수유형"]).reset_index().rename(columns={"index": "변수명"})

    # 연속형 변수 리스트 생성
    df_num_list = (
        df_var_list[
            (df_var_list["변수유형"] != 'object')
            & ~(df_var_list["변수명"].isin(dict_json.get("v_exc_num", [])))
        ]
    )    
         
    # 범주형 변수 리스트 생성
    df_char_list = (
        df_var_list[
            (df_var_list["변수유형"] == 'object')
            & ~(df_var_list["변수명"].isin(dict_json.get("v_exc_char", [])))
        ]
    )
    
    # 연속형 변수 결측값에 대해 일괄적으로 0으로 대체, 범주형 변수 결측값에 대해 일괄적으로 '기타'로 대체
    df_mart_2 = df_mart_1.copy()
    df_mart_2[df_num_list["변수명"]] = df_mart_2[df_num_list["변수명"]].fillna(0)
    df_mart_2[df_char_list["변수명"]] = df_mart_2[df_char_list["변수명"]].fillna('기타')
    
    # 결측치 변수 리스트 생성
    df_missing_list = pd.DataFrame(df_mart_1.isnull().sum(), columns=["변수개수"]).reset_index().rename(columns={"index": "변수명"})

    # 결측치 처리한 변수에 대한 설명을 담을 새로운 컬럼 추가
    df_missing_list["처리방법"] = ""
    
    df_missing_list = (
        df_missing_list[
            (df_missing_list["변수개수"] != 0)
            & ~(df_missing_list["변수명"].isin(dict_json.get("v_exc_num", []))
            | df_missing_list["변수명"].isin(dict_json.get("v_exc_char", [])))
        ]
    ) 
    
    # 결측치 변수 리스트 생성
    list_missing = df_missing_list["변수명"].tolist()
    list_num = df_num_list["변수명"].tolist()
    list_char = df_char_list["변수명"].tolist()
    
    # 결측치 처리 방법 기록
    for v_missing in list_missing:
        if v_missing in list_num:
            df_missing_list.loc[df_missing_list["변수명"] == v_missing, "처리방법"] = '0'
        elif v_missing in list_char:
            df_missing_list.loc[df_missing_list["변수명"] == v_missing, "처리방법"] = '기타'
    
    return df_mart_2, df_missing_list

# dict_json 정의
dict_json = {}

# 결측치 처리 수행
df_mart, df_missing_info = func_pps_missing(df_mart, dict_json)

# 결과 출력
print("결측치 처리 완료된 df_mart:")
print(df_mart)
print("\n결측치 처리 정보:")
print(df_missing_info)

결측치 처리 완료된 df_mart:
          고객ID   나이  성별  수익자여부  CB신용평점  CB신용등급  두낫콜여부     운전코드명 성별코드  피보험자여부  \
0        25226  60대  여성      1    0.00     0.0    1.0  승용차(자가용)   02       1   
1        95256  40대  남성      1    0.00     6.0    0.0  화물차(자가용)   02       1   
2        14751  40대  여성      1    0.00     0.0    1.0        기타   02       1   
3         4478  50대  여성      1    0.00     6.0    0.0      운전안함   01       1   
4       100000  50대  여성      0  245.26     0.0    0.0      운전안함   02       1   
...        ...  ...  ..    ...     ...     ...    ...       ...  ...     ...   
119995    7665  20대  여성      1    0.00     0.0    0.0        기타   01       1   
119996    6021  50대  여성      0    0.00     0.0    1.0      운전안함   01       1   
119997   50805  50대  남성      1    0.00     0.0    1.0        기타   02       1   
119998  100000  20대  여성      1    0.00     0.0    1.0  승용차(자가용)   02       0   
119999    6674  50대  여성      1    0.00     0.0    0.0        기타   01       1   

        ...  변액보유여부